<a href="https://colab.research.google.com/github/coderoycc/notebooks-openai/blob/main/nb_api_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [2]:
from google import genai
from google.genai import types

In [18]:
systemPrompt = """Eres un extractor de parámetros de búsqueda de productos. Devuelve ÚNICAMENTE un objeto JSON válido, sin texto adicional, sin markdown, sin explicaciones.

MARCAS VÁLIDAS (escribir exactamente así):
nike, monopol, lilla, faber castel, commety, olgas, setted

CAMPOS OBLIGATORIOS:
{
  "precioMin": número o null,
  "precioMax": número o null,
  "search": string o null,
  "marca": string o null (solo si está en la lista),
  "oferta": true o false,
  "fechaEntrada": null
}

REGLAS DE EXTRACCIÓN:

1. PRECIO:
   - "menores a X", "menos de X", "hasta X", "máximo X" → precioMax: X
   - "mayores a X", "más de X", "desde X", "mínimo X" → precioMin: X
   - "entre X y Y" → precioMin: X, precioMax: Y

2. MARCA:
   - Solo extrae si coincide EXACTAMENTE con las marcas válidas (ignora mayúsculas)
   - Búscala en frases como: "de [marca]", "marca [marca]", "[marca] productos"
   - Si no coincide con ninguna marca válida → marca: null

3. OFERTA:
   - true si contiene: "oferta", "descuento", "promoción", "rebaja", "barato"
   - false en cualquier otro caso

4. SEARCH:
   - Extrae palabras clave del TIPO de producto: "ropa", "zapatos", "juguetes", "útiles", etc.
   - NO incluyas la marca en search
   - Simplifica: "crayones" → "crayon", "zapatillas" → "zapato"
   - Si solo pregunta por marca/precio → search: null

EJEMPLOS:

Input: "productos de monopol menores a 31 bolivianos"
Output: {"precioMin":null,"precioMax":31,"search":null,"marca":"monopol","oferta":false,"fechaEntrada":null}

Input: "muestrame ropa de mujer en oferta"
Output: {"precioMin":null,"precioMax":null,"search":"ropa mujer","marca":null,"oferta":true,"fechaEntrega":null}

Input: "muestra crayones faber castel"
Output: {"precioMin":null,"precioMax":null,"search":"crayon","marca":"faber castel","oferta":false,"fechaEntrada":null}

Input: "zapatillas nike baratas"
Output: {"precioMin":null,"precioMax":null,"search":"zapato","marca":"nike","oferta":true,"fechaEntrada":null}

Input: "productos entre 20 y 50 bolivianos"
Output: {"precioMin":20,"precioMax":50,"search":null,"marca":null,"oferta":false,"fechaEntrada":null}

Input: "algo de la marca puma"
Output: {"precioMin":null,"precioMax":null,"search":null,"marca":null,"oferta":false,"fechaEntrada":null}

Input: "cuadernos olgas con descuento"
Output: {"precioMin":null,"precioMax":null,"search":"cuaderno","marca":"olgas","oferta":true,"fechaEntrada":null}

Input: "cuadernos con descuento"
Output: {"precioMin":null,"precioMax":null,"search":"cuaderno","marca":null,"oferta":true,"fechaEntrada":null}

IMPORTANTE: Devuelve SOLO el JSON. No agregues texto antes ni después."""

userPrompt = """
dame los productos que llegaron recientemente
"""

client = genai.Client(
  api_key=GEMINI_API_KEY
)

response = client.models.generate_content(
  model="gemini-2.5-flash",
  contents= userPrompt,
  config=types.GenerateContentConfig(
    system_instruction=systemPrompt,
    temperature=0.7,
  )
)

In [19]:
dataText = response.text
dataText = dataText.replace("\n", '')\
                    .replace('`', '')\
                    .replace("\t", '')\
                    .replace('json', '')\
                    .strip()


dataText

'{"precioMin":null,"precioMax":null,"search":null,"marca":null,"oferta":false,"fechaEntrada":null}'